## Find similar content from MLOps.Community meetups based on search. 

For context, I wanted to do that for all MLOps.community Meetups worldwide but the API for Meetup.com isn't free, therefore I decided to only do it for the ones of Berlin, I had to manually copy the data as well as Meetup wants you to be logged in to access the details of a Meetup.

I am using Milvus to perform a Similarity Search with the content of the MLOps.community meetups. 

I also use ChatGPT to summarise the content of the most similar meetups.

### Install all Libraries 

In [1]:
! poetry add milvus==2.2.14 pymilvus openai python-dotenv sentence-transformers

Using version ^2.3.7 for pymilvus
Using version ^1.14.1 for openai
Using version ^1.0.1 for python-dotenv
Using version ^2.5.1 for sentence-transformers

Updating dependencies
Resolving dependencies... (3.9s)

Package operations: 0 installs, 4 updates, 0 removals

  - Updating fsspec (2024.2.0 -> 2024.3.1): Pending...
  - Updating fsspec (2024.2.0 -> 2024.3.1): Downloading... 0%
  - Updating fsspec (2024.2.0 -> 2024.3.1): Downloading... 100%
  - Updating fsspec (2024.2.0 -> 2024.3.1): Installing...
  - Updating fsspec (2024.2.0 -> 2024.3.1)
  - Updating pyarrow (15.0.1 -> 15.0.2): Pending...
  - Updating setuptools (69.1.0 -> 69.2.0): Pending...
  - Updating setuptools (69.1.0 -> 69.2.0): Downloading... 0%
  - Updating setuptools (69.1.0 -> 69.2.0): Downloading... 0%
  - Updating pyarrow (15.0.1 -> 15.0.2): Installing...
  - Updating setuptools (69.1.0 -> 69.2.0): Downloading... 0%
  - Updating setuptools (69.1.0 -> 69.2.0): Downloading... 100%
  - Updating setuptools (69.1.0 -> 69.2.0

In [5]:
! pip install milvus==2.2.14 pymilvus openai python-dotenv sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 12.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.8 MB/s eta 0:00:0031m13.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.7/387.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.4 MB/s eta 0:00:0031m13.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 12.8 MB/s eta 0:00:0031m13.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 11.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.6 MB/s eta 0:00:0031m13.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 11.7 MB/s eta 0:0

In [2]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


#### I encountered some problems with using `default_server` from Milvus so I am using the `debug_server`. 

In [3]:
from pymilvus import connections, db
from milvus import default_server

conn = connections.connect(uri=f'http://localhost:{default_server.listen_port}')

db.list_database()


['default']

## Create the Milvus collection needed to perform a similarity search

In [5]:
from pymilvus import utility

if utility.has_collection("mlops_meetups_berlin"):
    utility.drop_collection("mlops_meetups_berlin")

In [6]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

# object should be inserted in the format of title, date, content, content embedding
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="date", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=10000),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384)
]
schema = CollectionSchema(fields=fields)
collection = Collection(name="mlops_meetups_berlin", schema=schema)

In [7]:
collection.create_index(field_name="embedding")
collection.load()

In [8]:
from sentence_transformers import SentenceTransformer

transformer = SentenceTransformer('all-MiniLM-L6-v2')

/Users/stephen/Library/Caches/pypoetry/virtualenvs/mlops-community-vector-search-DDGVbuBb-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import pandas as pd 

df = pd.read_csv('data/data_meetup.csv')
df.head()

,title,date,content
0,MLOps.community Berlin Meetup 05,"Thursday, September 14, 2023",Details\nHello Community!\n\nGet your calendar...
1,MLOps.community Berlin 04: Pre-event Women+ In...,"Thursday, June 29, 2023",Details\nHello Community!\n\nGet your calendar...
2,[In-person] Vector DB & LLM Hackathon,"Saturday, June 17, 2023","Details\nJoin us on Saturday, June 17 in Berli..."
3,MLOps.community Berlin Meetup 03,"Thursday, February 2, 2023",Details\nHello Community!\n\nOn February 2nd w...
4,MLOps.community Berlin Meetup 02,"Thursday, October 6, 2022",Details\nHello Community!\n\nOn October 6th we...


In [11]:
content_detail = df['content']
content_detail = content_detail.tolist()
embeddings = [transformer.encode(c) for c in content_detail]

#### Embed the content to be able to search it.

In [12]:
df['embedding'] = embeddings
df.head()

,title,date,content,embedding
0,MLOps.community Berlin Meetup 05,"Thursday, September 14, 2023",Details\nHello Community!\n\nGet your calendar...,"[-0.12466437, -0.050888978, -0.011999283, 0.07..."
1,MLOps.community Berlin 04: Pre-event Women+ In...,"Thursday, June 29, 2023",Details\nHello Community!\n\nGet your calendar...,"[-0.0037062387, -0.04097239, 0.040712878, 0.04..."
2,[In-person] Vector DB & LLM Hackathon,"Saturday, June 17, 2023","Details\nJoin us on Saturday, June 17 in Berli...","[-0.036718927, -0.07346592, 0.009544252, 0.016..."
3,MLOps.community Berlin Meetup 03,"Thursday, February 2, 2023",Details\nHello Community!\n\nOn February 2nd w...,"[-0.03421999, -0.063562274, -0.011161464, -0.0..."
4,MLOps.community Berlin Meetup 02,"Thursday, October 6, 2022",Details\nHello Community!\n\nOn October 6th we...,"[-0.014285266, -0.08103279, -0.023105947, 0.06..."


In [13]:
collection.insert(data=df)

(insert count: 6, delete count: 0, upsert count: 0, timestamp: 448470526131961858, success count: 6, err count: 0)

In [19]:
search_terms = "The speaker speaks about Open Source and ML Platform"
search_data = [transformer.encode(search_terms)] # Must be a list.

In [20]:
res = collection.search(
    data=search_data,  # Embedded search value
    anns_field="embedding",  # Search across embeddings
    param={"metric_type": "IP"},
    limit = 3,  # Limit to top_k results per search
    output_fields=["title", "content"]  # Include title field in result
)

In [21]:
def summarise_meetup_content(content: str) -> str: 
    response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
          "role": "system",
          "content": "Summarize content you are provided with."
        },
        {
          "role": "user",
          "content": f"{content}"
        }
    ],
        temperature=0,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    summary = response.choices[0].message.content
    return summary

In [25]:
for hits_i, hits in enumerate(res):
    print("Search Terms:", search_terms)
    print("Results:")
    for hit in hits:
        content_test = hit.entity.get("content")
        print(hit.entity.get("title"), "----", hit.distance)
        print(f'{summarise_meetup_content(hit.entity.get("content"))} \n')
    print()

Search Terms: The speaker speaks about Open Source and ML Platform
Results:
First MLOps.community Berlin Meetup ---- 0.5537543296813965
The MLOps.community meetup in Berlin on June 30th will feature a main talk by Stephen Batifol from Wolt on Scaling Open-Source Machine Learning. The event will also include lightning talks, networking, and food/drinks. The agenda includes Stephen's talk, Q&A session, lightning talks, and socializing. Attendees can sign up for lightning talks on Meetup.com. The event is in collaboration with neptune.ai. 

MLOps.community Berlin 04: Pre-event Women+ In Data and AI Festival ---- 0.46235063672065735
The MLOps.community Berlin is hosting a special edition event on June 29th and 30th at Thoughtworks. This event serves as a warm-up for the Women+ In Data and AI festival. The meetup will feature speakers Fiona Coath discussing surveillance capitalism and Magdalena Stenius talking about the carbon footprint of machine learning. The agenda includes talks, lightn